In [1]:
import re
import pandas as pd

In [60]:
with open("Lærkeskellet.txt","r") as f:
    body = f.read()
    print(body)


Salgspriser - alle boliger - Lærkeskellet, 2000

Adresse                                         Type           Aktuel
                                                               m²
Lærkeskellet 18, 2000 Frederiksberg             Ejerlejlighed
                                                4.575.000      140
Fri handel     15-05-2018                       1.026.500      138

Fri handel     27-07-1994                       Ejerlejlighed  140
                                                4.800.000      140
Lærkeskellet 6, 2000 Frederiksberg              3.070.000
                                                907.133        140
Fri handel     16-11-2017                                      140
                                                Ejerlejlighed  140
Fri handel     31-01-2012                       3.495.000      138
                                                               138
Fri handel     18-11-1994                       Ejerlejlighed
                             

In [29]:

body

'Salgspriser - alle boliger - Lindeskellet, 2000\n\nAdresse                                         Type           Aktuel\n                                                               m²\nLindeskellet 18, st., 2000 Frederiksberg        Ejerlejlighed\n                                                4.345.000      122\nFri handel     20-10-2019                       400.000\n                                                1.985.000      62\nI øvrigt       21-12-2011                       725.000\n                                                               130\nFri handel     05-09-2002                       Ejerlejlighed\n                                                2.199.000      62\nFri handel     21-06-1993                       1.530.000\n                                                483.000        122\nLindeskellet 16, 2., 2000 Frederiksberg                        123\n                                                Ejerlejlighed  124\nFri handel     29-01-2018            

In [40]:
re.findall('Lindeskellet \d{1,3}, [0-9a-z.]*, 2000', body)

['18, st., 2000',
 '16, 2., 2000',
 '10, 2., 2000',
 '12, st., 2000',
 '2, st., 2000',
 '12, 2., 2000',
 '16, st., 2000',
 '6, 2., 2000',
 '18, 2., 2000',
 '2, 2., 2000']

In [34]:
addresses = re.findall('Lindeskellet \d{1,2},', body)
square_meters = re.findall(' \d{2,3}\n', body)
dates = re.findall('\d\d-\d\d-\d\d\d\d', body)
prices = re.findall('[\d{1,3}.]*\d{3}.\d{3}', body)

I need to be able to find the addresses first and then find the dates for those addesses. Otherwise I can't line up the address, salesdate. 

The salesdate and price have the exact same number of rows, so those already lign up. But one or mole salesdates and prices can be part of the same address.

In [97]:
streetnames = [
    'Ahornskellet',
    'Askeskellet',
    'Birkeskellet',
    'Bøgeskellet',
    'Egeskellet',
    'Lindeskellet',
    'Lærkeskellet',
    'Pileskellet',
    'Plantanskellet',
    'Poppelskellet',
    'Tjørneskellet'
]
rows = []
for street in streetnames:
    with open(f"{street}.txt","r") as f:
        body = f.read()
    addresses = re.findall(f'{street} ' + '\d{1,3}, [0-9a-z., ]*2000', body)
    address_chuncks = re.split(f'{street} '+ '\d{1,2},', body)

    prices = re.findall('[\d{1,3}.]*\d{3}.\d{3}', body)
    square_meters = re.findall(' \d{2,3}\n', body)
    
    
    price_cnt = 0
    for address, m2, text in zip(addresses, square_meters, address_chuncks[1:]):
        m2 = int(m2.replace('\n',''))
        dates = re.findall('\d\d-\d\d-\d\d\d\d', text)
        for date in dates:
            try:
                price = prices[price_cnt]
                price = int(price.replace('.',''))
                
            except:
                break

            m2_price = round(price/m2)
            rows.append([address, date, price, m2, m2_price])
            price_cnt += 1

df = pd.DataFrame(columns=[
        'address',
        'salesDate',
        'price',
        'm2',
        'm2Price'
    ],
    data=rows)
len(df)

301

In [98]:
df

,address,salesDate,price,m2,m2Price
0,"Ahornskellet 14, 2., 2000",22-10-2019,2450000,62,39516
1,"Ahornskellet 14, 2., 2000",15-12-2011,1530000,62,24677
2,"Ahornskellet 14, 2., 2000",02-04-2008,1425000,62,22984
3,"Ahornskellet 6, 2., 2000",06-10-2019,3900000,111,35135
4,"Ahornskellet 6, 2., 2000",15-11-2005,3295000,111,29685
5,"Ahornskellet 16, st., 2000",22-04-2018,4225000,123,34350
6,"Ahornskellet 16, st., 2000",09-05-2014,3000000,123,24390
7,"Ahornskellet 16, st., 2000",02-11-2012,2900000,123,23577
8,"Ahornskellet 16, st., 2000",18-04-2007,3000000,123,24390
9,"Ahornskellet 6, st., 2000",24-10-2017,4100000,123,33333
